In [ ]:
from keras.layers import Dropout, Dense, GRU, Embedding
from keras.models import Sequential
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn import metrics
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import re
import nltk
import pandas as pd
from textblob import Word
import numpy as np
import csv
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras import layers
from keras.layers import  Dropout, Dense
from sklearn import metrics

Using TensorFlow backend.


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving bbc_preprocessed.csv to bbc_preprocessed.csv


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving glove.6B.50d.txt to glove.6B.50d.txt


In [ ]:
def loadData_Tokenizer(X_train, X_test,MAX_NB_WORDS=75000,MAX_SEQUENCE_LENGTH=500):
    np.random.seed(7)
    text = np.concatenate((X_train, X_test), axis=0)
    text = np.array(text)
    tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
    tokenizer.fit_on_texts(text)
    sequences = tokenizer.texts_to_sequences(text)
    word_index = tokenizer.word_index
    text = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
    print('Found %s unique tokens.' % len(word_index))
    indices = np.arange(text.shape[0])
    # np.random.shuffle(indices)
    text = text[indices]
    print(text.shape)
    X_train = text[0:len(X_train), ]
    X_test = text[len(X_train):, ]
    embeddings_index = {}
    f = open("glove.6B.50d.txt", encoding ="utf8")
    for line in f:

        values = line.split()
        word = values[0]
        try:
            coefs = np.asarray(values[1:], dtype='float32')
        except:
            pass
        embeddings_index[word] = coefs
    f.close()
    print('Total %s word vectors.' % len(embeddings_index))
    return (X_train, X_test, word_index,embeddings_index)

In [ ]:
def Build_Model_RNN_Text(word_index, embeddings_index, nclasses,  MAX_SEQUENCE_LENGTH=500, EMBEDDING_DIM=50, dropout=0.5):
    """
    def buildModel_RNN(word_index, embeddings_index, nclasses,  MAX_SEQUENCE_LENGTH=500, EMBEDDING_DIM=50, dropout=0.5):
    word_index in word index ,
    embeddings_index is embeddings index, look at data_helper.py
    nClasses is number of classes,
    MAX_SEQUENCE_LENGTH is maximum lenght of text sequences
    """

    model = Sequential()
    hidden_layer = 3
    gru_node = 32

    embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            if len(embedding_matrix[i]) != len(embedding_vector):
                print("could not broadcast input array from shape", str(len(embedding_matrix[i])),
                      "into shape", str(len(embedding_vector)), " Please make sure your"
                                                                " EMBEDDING_DIM is equal to embedding_vector file ,GloVe,")
                exit(1)
            embedding_matrix[i] = embedding_vector
    model.add(Embedding(len(word_index) + 1,
                                EMBEDDING_DIM,
                                weights=[embedding_matrix],
                                input_length=MAX_SEQUENCE_LENGTH,
                                trainable=True))


    print(gru_node)
    for i in range(0,hidden_layer):
        model.add(GRU(gru_node,return_sequences=True, recurrent_dropout=0.2))
        model.add(Dropout(dropout))
    model.add(GRU(gru_node, recurrent_dropout=0.2))
    model.add(Dropout(dropout))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(nclasses, activation='softmax'))


    model.compile(loss='sparse_categorical_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])
    return model

In [ ]:
data = pd.read_csv('bbc_preprocessed.csv')
#Label encoding
from sklearn import preprocessing 
  
# label_encoder object knows how to understand word labels. 
label_encoder = preprocessing.LabelEncoder() 
  
# Encode labels in column 'category'. 
data['category']= label_encoder.fit_transform(data['category']) 
data['category'].unique() 
x = data['text'].tolist()
y = data['category'].tolist()
x_train, x_test, Y_train, Y_test = train_test_split(x, y, test_size=0.50, random_state=1000)
X_train_Glove,X_test_Glove, word_index,embeddings_index = loadData_Tokenizer(x_train,x_test)

model_RNN = Build_Model_RNN_Text(word_index,embeddings_index, 5)

model_RNN.fit(X_train_Glove, Y_train,
                              validation_data=(X_test_Glove, Y_test),
                              epochs=20,
                              batch_size=128,
                              verbose=2)

predicted = model_RNN.predict_classes(X_test_Glove)

print(metrics.classification_report(Y_test, predicted))

Found 24799 unique tokens.
(2225, 500)
Total 400000 word vectors.
32
Train on 1780 samples, validate on 445 samples
Epoch 1/20
 - 36s - loss: 1.5860 - acc: 0.2702 - val_loss: 1.5173 - val_acc: 0.4225
Epoch 2/20
 - 31s - loss: 1.4249 - acc: 0.4163 - val_loss: 1.1724 - val_acc: 0.4247
Epoch 3/20
 - 31s - loss: 1.1496 - acc: 0.4966 - val_loss: 1.0253 - val_acc: 0.5551
Epoch 4/20
 - 31s - loss: 0.9609 - acc: 0.6455 - val_loss: 0.7826 - val_acc: 0.6944
Epoch 5/20
 - 31s - loss: 0.7921 - acc: 0.7129 - val_loss: 0.6242 - val_acc: 0.7640
Epoch 6/20
 - 31s - loss: 0.6315 - acc: 0.7781 - val_loss: 0.5492 - val_acc: 0.8022
Epoch 7/20
 - 31s - loss: 0.5180 - acc: 0.8247 - val_loss: 0.5046 - val_acc: 0.8112
Epoch 8/20
 - 32s - loss: 0.4296 - acc: 0.8697 - val_loss: 0.4486 - val_acc: 0.8449
Epoch 9/20
 - 31s - loss: 0.3464 - acc: 0.8938 - val_loss: 0.4035 - val_acc: 0.8629
Epoch 10/20
 - 32s - loss: 0.2820 - acc: 0.9096 - val_loss: 0.3577 - val_acc: 0.8899
Epoch 11/20
 - 31s - loss: 0.2456 - acc: 0.

In [ ]:
loss, accuracy = model_RNN.evaluate(X_train_Glove, Y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model_RNN.evaluate(X_test_Glove, Y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Training Accuracy: 0.9938
Testing Accuracy:  0.9326
